## 用CNN算法对图片进行分类

#### 下载并加载数据

In [1]:
import tensorflow as tf 
import numpy as np
import os
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("data/fashion", one_hot=True)     #下载并加载mnist数据
x = tf.placeholder(tf.float32, [None, 784])                        #输入的数据占位符
y_actual = tf.placeholder(tf.float32, shape=[None, 10])            #输入的标签占位符

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


In [2]:
class Dataset(object):
    def __init__(self, dtype='uint8', is_row_iamge=False):
        '''数据集
        
        Args:
            dtype: uint8 或 float32，uint8：每个像素值的范围是[0, 255];float32像素值范围是[0., 1.]
            is_row_image: 是否将3维图片展开成1维
        '''
        images = np.fromfile('./images/test_image.bin', dtype=np.uint8).reshape(-1, 28, 28, 1)
        print(images.shape)
        if dtype == 'uint8':
            self.images = images
        else:
            images = images.astype(np.float32) / 255.
            self.images = images
        if is_row_iamge:
            self.images = images.reshape([-1, 784])
        self.num_of_images = 6500
        self.offset = 0
        print('共6500张图片')

    def next_batch(self, batch_size=50):
        # 返回False表示以及没有样本
        # 注意：最后一个批次可能不足batch_size 所以推荐选择6500可以整除的batch_size
        if (self.offset + batch_size) <= self.num_of_images:
            self.offset += batch_size
            return self.images[self.offset-batch_size : self.offset]
        elif self.offset < self.num_of_images:
            return self.images[self.offset : ]
        else:
            False

In [3]:
'''
# 参数保存目录
FLAGS = tf.app.flags.FLAGS
# Basic model parameters.
tf.app.flags.DEFINE_string('cnn_path', '/home/bigwork/', """存放模型的目录""")

tf.app.flags.DEFINE_string('cnn_parameters', 'mnist',"""模型的名称""")
'''

'\n# 参数保存目录\nFLAGS = tf.app.flags.FLAGS\n# Basic model parameters.\ntf.app.flags.DEFINE_string(\'cnn_path\', \'/home/bigwork/\', """存放模型的目录""")\n\ntf.app.flags.DEFINE_string(\'cnn_parameters\', \'mnist\',"""模型的名称""")\n'

#### 定义四个函数，分别用于初始化权值W，初始化偏置项b, 构建卷积层和构建池化层。

In [4]:
#定义一个函数，用于初始化所有的权值 W
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

#定义一个函数，用于初始化所有的偏置项 b
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)
  
#定义一个函数，用于构建卷积层
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#定义一个函数，用于构建池化层
def max_pool(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

In [5]:
'''
#模型文件所在的文件夹，是否存在，如果不存在，则创建文件夹
ckpt = tf.train.latest_checkpoint(FLAGS.my_list)
if not ckpt:
    if not os.path.exists(FLAGS.my_list):
        os.mkdir(FLAGS.my_list)
X_ = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
'''

'\n#模型文件所在的文件夹，是否存在，如果不存在，则创建文件夹\nckpt = tf.train.latest_checkpoint(FLAGS.my_list)\nif not ckpt:\n    if not os.path.exists(FLAGS.my_list):\n        os.mkdir(FLAGS.my_list)\nX_ = tf.placeholder(tf.float32, [None, 784])\ny_ = tf.placeholder(tf.float32, [None, 10])\n'

#### 构建网络。整个网络由两个卷积层（包含激活层和池化层），一个全连接层，一个dropout层和一个softmax层组成。 

In [6]:
#构建网络
x_image = tf.reshape(x, [-1,28,28,1])         #转换输入数据shape,以便于用于网络中
W_conv1 = weight_variable([5, 5, 1, 32])      
b_conv1 = bias_variable([32])       
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)     #第一个卷积层
h_pool1 = max_pool(h_conv1)                                  #第一个池化层

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
h_pool2 = max_pool(h_conv2)                                   #第二个池化层

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])              #reshape成向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)    #第一个全连接层

keep_prob = tf.placeholder("float") 
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层

#### 训练数据

In [8]:
cross_entropy = -tf.reduce_sum(y_actual*tf.log(y_predict))     #交叉熵
train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(cross_entropy)    #梯度下降法

correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y_actual,1))    
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))                 #精确度计算
sess=tf.InteractiveSession()                          
sess.run(tf.initialize_all_variables())
for i in range(25000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:                  #训练100次，验证一次
    train_acc = accuracy.eval(feed_dict={x:batch[0], y_actual: batch[1], keep_prob: 1.0})
    print ('step ',i, 'training accuracy ',train_acc)
    train_step.run(feed_dict={x: batch[0], y_actual: batch[1], keep_prob: 0.5})


check_point_path = '/home/bigwork/' # 保存好模型的文件路径
ckpt = tf.train.get_checkpoint_state(checkpoint_dir=check_point_path)
    #saver.restore(sess,ckpt.model_checkpoint_path)

Y = np.zeros(6500)
#X_batch, y_batch = mnist.test.next_batch(batch_size=10000)
images = Dataset(dtype='float32',is_row_iamge=True)
for j in range(130):
    X_batch = images.next_batch(batch_size=50)
    X_batch = X_batch.reshape((50, 784))
    Ytemp = y_predict.eval(feed_dict={x : X_batch[:], keep_prob: 1.0})
    for i in range(50):
        #生成0-9标签
        Y[j*50+i] = np.argmax(Ytemp[i])
    # print(Y[i])
# print("test accuracy %g" % accuracy.eval(feed_dict={X_: X_batch, y_: y_batch, keep_prob: 1.0}))
fp = open("test0.txt", "w+")
for i in range(6500):
    fp.write(str(int(Y[i]))+"\n")
fp.close()




Instructions for updating:
Use `tf.global_variables_initializer` instead.
step  0 training accuracy  0.3
step  100 training accuracy  0.12
step  200 training accuracy  0.18
step  300 training accuracy  0.06
step  400 training accuracy  0.02
step  500 training accuracy  0.14
step  600 training accuracy  0.24
step  700 training accuracy  0.24
step  800 training accuracy  0.24
step  900 training accuracy  0.28
step  1000 training accuracy  0.24
step  1100 training accuracy  0.18
step  1200 training accuracy  0.46
step  1300 training accuracy  0.46
step  1400 training accuracy  0.38
step  1500 training accuracy  0.5
step  1600 training accuracy  0.52
step  1700 training accuracy  0.6
step  1800 training accuracy  0.6
step  1900 training accuracy  0.52
step  2000 training accuracy  0.48
step  2100 training accuracy  0.48
step  2200 training accuracy  0.54
step  2300 training accuracy  0.78
step  2400 training accuracy  0.48
step  2500 training accuracy  0.66
step  2600 training accuracy  0.

step  22400 training accuracy  0.86
step  22500 training accuracy  0.82
step  22600 training accuracy  0.82
step  22700 training accuracy  0.86
step  22800 training accuracy  0.8
step  22900 training accuracy  0.76
step  23000 training accuracy  0.76
step  23100 training accuracy  0.84
step  23200 training accuracy  0.7
step  23300 training accuracy  0.84
step  23400 training accuracy  0.76
step  23500 training accuracy  0.8
step  23600 training accuracy  0.9
step  23700 training accuracy  0.84
step  23800 training accuracy  0.82
step  23900 training accuracy  0.9
step  24000 training accuracy  0.76
step  24100 training accuracy  0.84
step  24200 training accuracy  0.82
step  24300 training accuracy  0.92
step  24400 training accuracy  0.72
step  24500 training accuracy  0.76
step  24600 training accuracy  0.8
step  24700 training accuracy  0.84
step  24800 training accuracy  0.84
step  24900 training accuracy  0.84
(6500, 28, 28, 1)
共6500张图片


'\n#\u3000预测值中最大值（１）即分类结果，是否等于原始标签中的（１）的位置。argmax()取最大值所在的下标\nz = tf.argmax(y_predict, 1)\ntest_acc_sum = tf.Variable(0.0)\nbatch_acc = tf.placeholder(tf.float32)\nnew_test_acc_sum = tf.add(test_acc_sum, batch_acc)\nupdate = tf.assign(test_acc_sum, new_test_acc_sum)\nsaver=tf.train.Saver(max_to_keep=2)\n# 定义了变量必须要初始化，或者下面形式\nwith tf.Session() as sess:\n    sess.run(tf.global_variables_initializer())\n# 或者某个变量单独初始化 如：\n    check_point_path = \'/home/bigwork/\' # 保存好模型的文件路径\n    ckpt = tf.train.get_checkpoint_state(checkpoint_dir=check_point_path)\n    #saver.restore(sess,ckpt.model_checkpoint_path)\n\n    Y = np.zeros(10000)\n    X_batch, y_batch = mnist.test.next_batch(batch_size=10000)\n    Ytemp = y_predict.eval(feed_dict={x : X_batch[:1000], keep_prob: 1.0})\n    for i in range(1000):\n            #生成0-9标签\n        Y[i] = np.argmax(Ytemp[i])\n        # print(Y[i])\n    # print("test accuracy %g" % accuracy.eval(feed_dict={X_: X_batch, y_: y_batch, keep_prob: 1.0}))\nfp = open("test.